In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning);

import sys
sys.executable

'/home/mingrui/anaconda3/envs/py36_cu90/bin/python'

In [2]:
import os
import glob

# plotting
import PIL
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import Image
plt.rcParams['figure.figsize'] = [15, 15]

import deephistopath.wsi.slide as slide
import deephistopath.wsi.util as util
import deephistopath.wsi.filter as filter
import deephistopath.wsi.tiles as tiles
util.ADDITIONAL_NP_STATS = True

import py_wsi
import py_wsi.imagepy_toolkit as tk

import pandas as pd
pd.set_option('display.max_columns',None)

# Setup two classes

In [3]:
GLIOMA_DIR = '/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/'
GLIOMA_DB_DIR = ''
GLIOMA_DB_NAME = 'glioma_patch_db'

MENINGIOMA_DIR = '/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/'
MENINGIOMA_DB_DIR = ''
MENINGIOMA_DB_NAME = 'meningioma_patch_db'

PATCH_SIZE = 512
LEVEL = 15
OVERLAP = 0

In [4]:
turtle_glioma = py_wsi.Turtle(GLIOMA_DIR, GLIOMA_DB_DIR, GLIOMA_DB_NAME)
turtle_meningioma = py_wsi.Turtle(MENINGIOMA_DIR, MENINGIOMA_DB_DIR, MENINGIOMA_DB_NAME)

print("Total WSI images:    " + str(turtle_glioma.num_files))
print("LMDB name:           " + str(turtle_glioma.db_name))
print("File names:          " + str(turtle_glioma.files))

print("Total WSI images:    " + str(turtle_meningioma.num_files))
print("LMDB name:           " + str(turtle_meningioma.db_name))
print("File names:          " + str(turtle_meningioma.files))

glioma_file = turtle_glioma.files[0]
meningioma_file = turtle_meningioma.files[0]

5 WSI found in directory.
5 WSI found in directory.
Total WSI images:    5
LMDB name:           glioma_patch_db
File names:          ['132223.svs' '132217.svs' '132225.svs' '132214.svs' '132224.svs']
Total WSI images:    5
LMDB name:           meningioma_patch_db
File names:          ['122403.svs' '122401.svs' '122433.svs' '122468.svs' '122889.svs']


# Preprocess

In [22]:
def multiprocessing_pipeline(base_dir, prefix_name, turtle_files):
    # just change this
    slide.BASE_DIR = base_dir

    slide.SRC_TRAIN_DIR = slide.BASE_DIR
    slide.DEST_TRAIN_DIR = os.path.join(slide.BASE_DIR, "training_" + slide.DEST_TRAIN_EXT)
    slide.DEST_TRAIN_THUMBNAIL_DIR = os.path.join(slide.BASE_DIR, "training_thumbnail_" + slide.THUMBNAIL_EXT)
    slide.FILTER_DIR = os.path.join(slide.BASE_DIR, "filter_" + slide.DEST_TRAIN_EXT)
    slide.FILTER_THUMBNAIL_DIR = os.path.join(slide.BASE_DIR, "filter_thumbnail_" + slide.THUMBNAIL_EXT)
    slide.TILE_SUMMARY_DIR = os.path.join(slide.BASE_DIR, "tile_summary_" + slide.DEST_TRAIN_EXT)
    slide.TILE_SUMMARY_ON_ORIGINAL_DIR = os.path.join(slide.BASE_DIR, "tile_summary_on_original_" + slide.DEST_TRAIN_EXT) 
    slide.TILE_SUMMARY_THUMBNAIL_DIR = os.path.join(slide.BASE_DIR, "tile_summary_thumbnail_" + slide.THUMBNAIL_EXT)                                                                                                                     
    slide.TILE_SUMMARY_ON_ORIGINAL_THUMBNAIL_DIR = os.path.join(slide.BASE_DIR, "tile_summary_on_original_thumbnail_" + slide.THUMBNAIL_EXT)                                                                                             
    slide.TILE_SUMMARY_HTML_DIR = slide.BASE_DIR                                                                
    slide.TILE_DATA_DIR = os.path.join(slide.BASE_DIR, "tile_data")                                                                                             
    slide.TOP_TILES_DIR = os.path.join(slide.BASE_DIR, slide.TOP_TILES_SUFFIX + "_" +slide.DEST_TRAIN_EXT)                                                                         
    slide.TOP_TILES_THUMBNAIL_DIR = os.path.join(                                                                                                                                                                            
         slide.BASE_DIR,                                                                                                                                                                                                      
         slide.TOP_TILES_SUFFIX +                                                                                                                                                                                             
         "_thumbnail_" +                                                                                                                                                                                                
         slide.THUMBNAIL_EXT)                                                                                                                                                                                                 
    slide.TOP_TILES_ON_ORIGINAL_DIR = os.path.join(                                                                                                                                                                          
         slide.BASE_DIR,                                                                                                                                                                                                      
         slide.TOP_TILES_SUFFIX +                                                                                                                                                                                             
         "_on_original_" +                                                                                                                                                                                              
         slide.DEST_TRAIN_EXT)                                                                                                                                                                                                
    slide.TOP_TILES_ON_ORIGINAL_THUMBNAIL_DIR = os.path.join(                                                                                                                                                                
         slide.BASE_DIR, slide.TOP_TILES_SUFFIX + "_on_original_thumbnail_" + slide.THUMBNAIL_EXT)                                                                                                                                        
    slide.TILE_DIR = os.path.join(slide.BASE_DIR, "tiles_" + slide.DEST_TRAIN_EXT)                                                                                                                                                       
    slide.STATS_DIR = os.path.join(slide.BASE_DIR, "svs_stats")  

    slide.SLIDE_NAMES = turtle_files
    slide.TRAIN_PREFIX = prefix_name
    
    slide.multiprocess_training_slides_to_images()
    print('=====================================')
    filter.multiprocess_apply_filters_to_images()
    print('=====================================')
    tiles.multiprocess_filtered_images_to_tiles()

In [23]:
multiprocessing_pipeline(GLIOMA_DIR, 'glioma-', turtle_glioma.files)

/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132225.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132214.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132224.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132217.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132223.svs
Opening Slide #4: /media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132214.svs
Opening Slide #3: /media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132225.svs
Opening Slide #5: /media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132224.svs
Opening Slide #2: /media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132217.svs
Opening Slide #1: /media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/132223.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/IDH_sample/
Number of processes: 5
Number of training images: 5
Task #1: Process slid

In [24]:
multiprocessing_pipeline(MENINGIOMA_DIR, 'meningioma-', turtle_meningioma.files)

/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122401.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122889.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122433.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122403.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122468.svs
Opening Slide #2: /media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122401.svs
Opening Slide #4: /media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122468.svs
Opening Slide #3: /media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122433.svs
Opening Slide #5: /media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122889.svs
Opening Slide #1: /media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_sample/122403.svs
/media/disk1/mingrui-projects/python-wsi-preprocessing/Meningioma_samp